In [1]:
from qiskit import QuantumCircuit
from qiskit.transpiler import CouplingMap
import random
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes.routing.sabre_swap_v0_20_      import SabreSwap 
from qiskit.transpiler.passes import ApplyLayout, FullAncillaAllocation, \
                                     EnlargeWithAncilla
from qiskit.transpiler.passes.layout.sabre_layout    import SabreLayout
from qiskit.qasm2 import dump 
random.seed(42)

In [2]:
def add_randomized_non_overlapping_layer(qc, coupling_list, gate_type='cx'):
    # Shuffle the coupling list to ensure random exploration
    shuffled_coupling_list = coupling_list.copy()
    random.shuffle(shuffled_coupling_list)
    
    selected_edges = []
    used_qubits = set()
    
    # Select non-overlapping edges from the shuffled list
    for edge in shuffled_coupling_list:
        if edge[0] not in used_qubits and edge[1] not in used_qubits:
            selected_edges.append(edge)
            used_qubits.update(edge)
    
    # Randomly reduce the number of selected edges to between half and all of them
    min_edges = len(selected_edges) // 2 if len(selected_edges) % 2 == 0 else (len(selected_edges) // 2) + 1
    num_edges_to_include = random.randint(min_edges, len(selected_edges))
    edges_to_include = random.sample(selected_edges, num_edges_to_include)
    
    # Add CNOT or SWAP gates for each selected edge to the existing quantum circuit
    for edge in edges_to_include:
        if gate_type == 'cx':
            qc.cx(edge[0], edge[1])
        elif gate_type == 'swap':
            qc.swap(edge[0], edge[1])
        else:
            raise ValueError('Invalid gate type')
        
def create_parallel_circuit(num_qubits, num_layers, coupling_map):
    # Create a quantum circuit with the given number of qubits
    qc = QuantumCircuit(num_qubits)
    coupling_list = list(coupling_map.get_edges())
    
    # Add layers of randomized non-overlapping CNOT gates and SWAP gates
    for _ in range(num_layers):
        add_randomized_non_overlapping_layer(qc, coupling_list, gate_type='cx')
        add_randomized_non_overlapping_layer(qc, coupling_list, gate_type='swap')
    
    return qc

def apply_swaps_and_get_matching_circuit(qc):
    # Initialize a new quantum circuit with the same number of qubits
    new_qc = QuantumCircuit(qc.num_qubits)
    
    # Mapping of qubit indices to their positions after swaps
    qubit_map = {i: i for i in range(qc.num_qubits)}
    
    # Iterate through the circuit instructions in reverse order
    for instr, qargs, _ in reversed(qc.data):
        if instr.name == 'swap':
            # Update the qubit mapping based on the swap
            qubit_map[qargs[0]._index], qubit_map[qargs[1]._index] = qubit_map[qargs[1]._index], qubit_map[qargs[0]._index]
        elif instr.name == 'cx':
            # Apply the current qubit mapping to the CNOT gate and add it to the new circuit
            new_qc.cx(qubit_map[qargs[0]._index], qubit_map[qargs[1]._index])
    
    # Reverse the order of gates in the new circuit to reflect the original order
    new_qc = new_qc.reverse_ops()
    
    return new_qc



In [3]:
num_qubits = 10
num_layers = 20
cm = CouplingMap.from_ring(num_qubits)


qc = create_parallel_circuit(num_qubits, num_layers, cm)

# Get the matching circuit after swaps are applied
matching_circuit = apply_swaps_and_get_matching_circuit(qc)

# Get transpiled circuit of the matching circuit
rp = SabreSwap(cm, seed=42)
lp = SabreLayout(cm, routing_pass=rp)

pm = PassManager([
    lp,
    FullAncillaAllocation(cm),
    EnlargeWithAncilla(),
    ApplyLayout(),
    rp
])

qc_transpiled = pm.run(matching_circuit)
qc_transpiled.draw()

# Print the original and modified circuits
print("Original Circuit:")
print(qc)
print("\nModified Circuit after applying SWAPs:")
print(matching_circuit.draw())
print("\nTranspiled Circuit of the modified circuit:")
print(qc_transpiled.draw())

Original Circuit:
                          ┌───┐             ┌───┐          ┌───┐               »
q_0: ───────■──────X──────┤ X ├───────────X─┤ X ├──X───────┤ X ├──X────■───────»
            │      │      └─┬─┘           │ └─┬─┘  │       └─┬─┘  │  ┌─┴─┐     »
q_1: ───────┼───X──┼────────┼────X────────┼───■────X────■────┼────X──┤ X ├─────»
     ┌───┐  │   │  │        │    │        │           ┌─┴─┐  │       └───┘┌───┐»
q_2: ┤ X ├──┼───X──┼───X────┼────X────────┼───────────┤ X ├──┼─────────X──┤ X ├»
     └─┬─┘  │      │   │    │       ┌───┐ │      ┌───┐└───┘  │         │  └─┬─┘»
q_3: ──■────┼───X──┼───X────┼───────┤ X ├─┼───X──┤ X ├──X────┼────■────X────■──»
     ┌───┐  │   │  │        │       └─┬─┘ │   │  └─┬─┘  │    │  ┌─┴─┐          »
q_4: ┤ X ├──┼───X──┼───■────┼────X────■───┼───X────■────X────┼──┤ X ├──X───────»
     └─┬─┘  │      │ ┌─┴─┐  │    │        │                  │  └───┘  │       »
q_5: ──■────┼──────┼─┤ X ├──┼────X────────┼────────■─────────┼─────────X────X──»
          

In [4]:
print("\nOriginal Circuit Depth:", qc.depth())
print("Decomposed Original Circuit Depth:", qc.decompose().depth())

print("\nTranspiled Circuit Depth:", qc_transpiled.depth())
print("Decomposed Transpiled Circuit Depth:", qc_transpiled.decompose().depth())

print("\nModified Circuit Depth:", matching_circuit.depth())



Original Circuit Depth: 38
Decomposed Original Circuit Depth: 78

Transpiled Circuit Depth: 53
Decomposed Transpiled Circuit Depth: 114

Modified Circuit Depth: 18


In [8]:
def create_parallel_circuits_set(num_qubits, num_layers_list, coupling_map, cm_label):
    # Create a set of quantum circuits with the given number of qubits and number of layers
    qc_list = [] # a list of tuples (original_circuit, matching_circuit)
    for num_layers in num_layers_list:
        qc_orig = create_parallel_circuit(num_qubits, num_layers, coupling_map)
        file_answer = f"experiment_setup/circuits/zou_circuits/answers/parallel_circuit_{num_qubits}_{num_layers}_{cm_label}.qasm"
        dump(qc_orig, file_answer)

        qc_match = apply_swaps_and_get_matching_circuit(qc_orig)
        file_problem = f"experiment_setup/circuits/zou_circuits/problems/parallel_circuit_{num_qubits}_{num_layers}_{cm_label}.qasm"
        dump(qc_match, file_problem)

        qc_list.append((qc_orig, qc_match))

        print("Done with num_layers = ", num_layers)
    return qc_list

In [6]:
num_qubits = 27
num_layers_list = range(1, 101)
cm = CouplingMap.from_ring(num_qubits)
cm_label = "ring_27"

In [9]:
qc_list = create_parallel_circuits_set(num_qubits, num_layers_list, cm, cm_label)

Done with num_layers =  1
Done with num_layers =  2
